In [82]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
PCA on top?

In [83]:
from helpers import async_cache_pages, urls_list

import os
import random
random.seed(0)

In [84]:
folders = ["general2"]

positives, negatives = urls_list(folders)
async_cache_pages(positives + negatives)

Error while downloading: https://www.amazon.com/Low-Price-With-Free-Shipping/. Status code: 404
Error while downloading: http://blog.openclassrooms.com/en/. Status code: 403
Error while downloading: https://about.udemy.com/careers/. Status code: 403
Error while downloading: https://about.udemy.com/blog/?ref=footer. Status code: 403
Error while downloading: https://business.udemy.com/?ref=ufb_header. Status code: 403
Error while downloading: https://www.amazon.com/. Status code: 503
Trying again to download 0 pages one at a time


In [85]:
# Same number of negative and positive examples
urls = positives + random.sample(negatives, len(positives))
labels = [True] * len(positives) + [False] * len(positives)

In [86]:
from features import construct_text_df

In [87]:
df = construct_text_df(urls, labels)

Skipped , was not cached
Skipped , was not cached
Skipped https://www.amazon.com/Low-Price-With-Free-Shipping/, was not cached
Skipped http://blog.openclassrooms.com/en/, was not cached
Skipped https://www.amazon.com/, was not cached


In [88]:
df.sample(5, random_state=1)

,label,visible_text
url,,
http://online.stanford.edu/course/crisis-code-teaching-crisis-management-skills-enhance-management-advanced-cardiac-life,True,Skip content Skip navigation ONLINE Connect us...
https://www.futurelearn.com/courses/the-genomics-era,True,Contact FutureLearn Support Skip main navigati...
https://www.pinterest.ch/pin/316448311309858339/,False,Pinterest using cookies help give best experie...
https://openclassrooms.com/courses/integrate-a-wordpress-theme,True,"Vous utilisez un navigateur obsolète, veuillez..."
https://openclassrooms.com/courses/write-object-oriented-ruby,True,"Vous utilisez un navigateur obsolète, veuillez..."


In [89]:
X = df["visible_text"]
y = df.label

In [90]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [91]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [92]:
X_counts = count_vect.fit_transform(X)

In [93]:
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [94]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [95]:
feats = count_vect.get_feature_names()
feats[10000:10000 + 10]

['karma',
 'kaszã',
 'kate',
 'katherine',
 'katie',
 'katy',
 'katyperry',
 'katyperryvevo',
 'katz',
 'kauf']

In [96]:
top_feats_in_doc(X_tfidf, feats, 0, 10)

,feature,tfidf
0,java,0.498376
1,video,0.355078
2,csv,0.224788
3,programming,0.216111
4,course,0.163338
5,reading,0.141775
6,duke,0.139201
7,grayscale,0.121040
8,data,0.120887
9,practice,0.115657


In [97]:
top_feats_in_doc(X_tfidf, feats, 1, 10)

,feature,tfidf
0,strategy,0.616469
1,video,0.194265
2,strategic,0.177828
3,copenhagen,0.176004
4,ä¹,0.156093
5,business,0.151174
6,ä½,0.133794
7,¾ç,0.133794
8,porter,0.124167
9,course,0.121118


In [98]:
top_feats_in_doc(X_tfidf, feats, 2, 10)

,feature,tfidf
0,antimicrobial,0.491490
1,resistance,0.491490
2,antimicrobials,0.229493
3,susceptibility,0.229362
4,lecture,0.168935
5,course,0.141743
6,bacteria,0.141227
7,video,0.126329
8,ast,0.123573
9,methods,0.123146


In [99]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [100]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [101]:
docs_new = ["course submit", "education learning", "python coursera", "submit button", "medical hospital", "news article"]
predicted = pipeline.predict(docs_new)

In [102]:
for doc, category in zip(docs_new, predicted):
    print(doc, "==>", category)

course submit ==> True
education learning ==> True
python coursera ==> True
submit button ==> True
medical hospital ==> True
news article ==> False


In [103]:
from features import analyse_classification

In [104]:
pipeline2 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

analyse_classification(X, y, pipeline2)

             precision    recall  f1-score   support

      False       0.94      0.48      0.64        33
       True       0.67      0.97      0.79        35

avg / total       0.80      0.74      0.72        68

[[16 17]
 [ 1 34]]
Accuracy: 0.7352941176470589


In [105]:
from sklearn.linear_model import LogisticRegression
pipeline3 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression()),
])

analyse_classification(X, y, pipeline3)

             precision    recall  f1-score   support

      False       0.96      0.76      0.85        33
       True       0.81      0.97      0.88        35

avg / total       0.88      0.87      0.87        68

[[25  8]
 [ 1 34]]
Accuracy: 0.8676470588235294


In [106]:
pipeline3.decision_function(X)

array([ 0.90892938,  0.49684893,  0.61440345,  0.72660173,  0.55371857,
        0.51677329,  0.52180738,  0.35586578,  0.10372384,  0.32057602,
        0.2091007 ,  0.25646052,  0.20707336,  0.86994594,  1.06542028,
        1.01197603,  0.91439378,  0.940258  ,  0.48720265,  0.83076799,
        0.83076799,  0.38845262,  0.51289682,  0.41307959,  0.44091173,
        0.42707329,  0.16712731,  0.32684353,  0.40953064,  0.47022817,
        0.24850751,  0.61989597,  0.39848188,  0.37428465,  0.44017145,
       -0.17053502,  0.55475676, -0.23720919,  0.51755137,  0.49759991,
        0.60359728,  0.45221218,  0.54758529,  0.40014779,  0.14863885,
        0.35728663,  0.56220259,  0.48717578,  0.78737965,  0.76825365,
        0.67097296,  0.51463243,  0.89199834,  0.65794815,  0.9080439 ,
        0.42810768,  0.91784811,  0.7584666 ,  0.83323325,  0.8911259 ,
        0.54119524,  0.28751129,  0.65651708,  0.53139298,  0.65328606,
        0.8597689 ,  0.86905277,  0.77978517,  0.70104944,  0.75

Problem ? 0.87 quite high. Overfitting because 6-7 pages in the same platform ?

In [107]:
new_urls_true = [
    "https://www.wallstreetprep.com/self-study-programs/oil-and-gas-modeling/",
    "https://www.wallstreetprep.com/self-study-programs/restructuring-modeling/",
    "https://www.wallstreetprep.com/self-study-programs/bank-and-fig-modeling/",
    "https://www.wallstreetprep.com/self-study-programs/adv-lbo-modeling/",
    "https://www.educba.com/course/online-investment-banking-training-courses/",
    "https://www.creativelive.com/courses/find-your-niche-and-build-your-family-photography-business-julia-kelleher",
    "https://www.creativelive.com/courses/lightroom-cc-photo-editing-the-complete-guide-ben-willmore",
    "https://www.creativelive.com/courses/portrait-compositing-from-start-to-finish-matt-kloskowski?via=class-list-collection_3",
    "https://onlinecoursemasters.com/",
    "https://ocw.mit.edu/courses/mathematics/18-01-single-variable-calculus-fall-2006/index.htm",
    "http://tutorial.math.lamar.edu/Classes/CalcI/CalcI.aspx",
    "https://www.khanacademy.org/math/calculus-home",
    "https://www.coursera.org/learn/calculus1",
    "https://www.simplilearn.com/blockchain-certification-training",
]

new_urls_false = [
    "https://twitter.com/?lang=en",
    "http://iamafoodblog.com/",
    "http://iamafoodblog.com/category/recipes/",
    "https://keepvid.com/sites/download-youtube-video.html",
    "https://en.wikipedia.org/wiki/Massive_open_online_course",
    "https://github.com/",
    "https://github.com/features",
    "https://news.ycombinator.com/",
    "https://www.schneems.com/2017/11/14/wtf-is-a-source-map/",
    "https://medium.freecodecamp.org/using-svg-as-placeholders-more-image-loading-techniques-bed1b810ab2c"
]


async_cache_pages(new_urls_true + new_urls_false)

Trying again to download 0 pages one at a time


In [108]:
# Not using these labels in the prediction but need them for df construction
new_labels_true = [True] * len(new_urls_true)
new_labels_false = [False] * len(new_urls_false)

df_true = construct_text_df(new_urls_true, new_labels_true)
df_false = construct_text_df(new_urls_false, new_labels_false)

X_new_true = df_true["visible_text"]
X_new_false = df_false["visible_text"]

In [109]:
pipeline3.predict(X_new_true)

array([ True, False,  True,  True, False, False, False, False,  True,
        True, False, False,  True,  True], dtype=bool)

In [110]:
pipeline3.decision_function(X_new_true)

array([ 0.01797621, -0.03439607,  0.08594231,  0.13512135, -0.06181718,
       -0.15405072, -0.00346216, -0.05705639,  0.90560204,  0.12373138,
       -0.12965627, -0.22083891,  0.21134179,  0.02016643])

In [111]:
pipeline3.predict(X_new_false)

array([False, False, False, False,  True, False, False, False, False, False], dtype=bool)

In [112]:
pipeline3.decision_function(X_new_false)

array([-0.43295055, -0.45166017, -0.5944089 , -0.51565106,  0.00631062,
       -0.32493084, -0.17016123, -0.80597136, -0.18449325, -0.2394083 ])

Yes, works well because of the duplication in the training set, but we see that there are almost no false positive, and the false negatives have a value close to zero in the decision function, meaning it is "unsure".

In [113]:
folders = ["general3"]

positives, negatives = urls_list(folders)
async_cache_pages(positives + negatives)

Error while downloading: http://blog.openclassrooms.com/en/. Status code: 403
Error while downloading: https://about.udemy.com/blog/?ref=footer. Status code: 403
Error while downloading: https://www.udemy.com/data-analysis-with-excel-pivot-tables/%20#does%20not%20work%20-%20403. Status code: 403
Error while downloading: https://business.udemy.com/?ref=ufb_header. Status code: 403
Error while downloading: https://about.udemy.com/careers/. Status code: 403
Trying again to download 0 pages one at a time


In [114]:
# Same number of negative and positive examples
urls = positives + random.sample(negatives, len(positives))
labels = [True] * len(positives) + [False] * len(positives)

In [115]:
df = construct_text_df(urls, labels)
X = df["visible_text"]
y = df.label

Skipped https://www.udemy.com/data-analysis-with-excel-pivot-tables/ #does not work - 403, was not cached
Skipped https://www.skillshare.com/classes/Low-Budget-Filmmaking-—-Tips-and-Tricks-for-an-Indie-Look/1077872333, was not cached
Skipped http://blog.creativelive.com, was not cached
Skipped https://about.udemy.com/careers/, was not cached
Skipped https://www.skillshare.com, was not cached
Skipped https://business.udemy.com/?ref=ufb_header, was not cached
Skipped http://blog.openclassrooms.com/en/, was not cached


In [116]:
# Multinomial NB
analyse_classification(X, y, pipeline2)

             precision    recall  f1-score   support

      False       1.00      0.24      0.39        54
       True       0.53      1.00      0.69        46

avg / total       0.78      0.59      0.53       100

[[13 41]
 [ 0 46]]
Accuracy: 0.59


In [128]:
from sklearn.neighbors import KNeighborsClassifier
pipeline4 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=5)),
])
analyse_classification(X, y, pipeline4)

             precision    recall  f1-score   support

      False       0.68      0.91      0.78        54
       True       0.82      0.50      0.62        46

avg / total       0.75      0.72      0.71       100

[[49  5]
 [23 23]]
Accuracy: 0.72


In [117]:
# Logistic regression
analyse_classification(X, y, pipeline3)

             precision    recall  f1-score   support

      False       0.97      0.72      0.83        54
       True       0.75      0.98      0.85        46

avg / total       0.87      0.84      0.84       100

[[39 15]
 [ 1 45]]
Accuracy: 0.84


In [118]:
from sklearn.externals import joblib
import os

In [119]:
joblib.dump(pipeline3, os.path.join('saved', 'models', 'log_reg_pipeline_general3.pkl'))

['saved/models/log_reg_pipeline_general3.pkl']

The best classifier for NLP is not the same as for Structural Features. Also, LogisticRegression is convenient because it gives a "decision_function" that can be used later on.